# Das Problem von Deutsch und verwandet Algorithmen
In diesem Jupyter Notebook werden ein paar Algorithmen vorgestellt, bei denen es darum geht auf besonders effiziente Weise Eigenschaften über eine Operation deren Details unbekannt sind zu bestimmen. Eine solche Operation, die quasi als abgeschlossenes Bauteil übergeben wird, wird auch als BlackBox bezeichnet. Im Fall von Quantencomputern werden Bauteile auch häufig als Quantenorakel bezeichnet. 

Zunächst wird ein Bauteil betrachtet, das nur mit einem Eingabe-Qubit arbeitet und es wird bestimmt, ob die Operation balanciert (eindeutig umkehrbar) oder nicht ist. 

Die beiden anderen Algorithmen bestimmen Eigenschaften eines Bauteils, dass beliebig viele Qubits entgegen nimmt und diese entweder auf $0$ oder $1$ abbildet. Alle diese Algorithmen haben gemeinsam, dass sie ein Hilfsbit benötigen.

## Vorbereitungen
Wie am Anfang jedes unserer Jupyter Notebooks werden einige Pakete geladen. (Siehe hierzu bspw. ["Einstieg in Qiskikt"](https://github.com/MarkEich96/QuantencomputingWiSe23-24/blob/main/Einstieg%20in%20Qiskit.ipynb) oder ["Zufallsgenerator"](https://github.com/MarkEich96/QuantencomputingWiSe23-24/blob/main/Zufallsgenerator.ipynb))

In [ ]:
import numpy as np
import qiskit
from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister)
from qiskit import(execute, Aer, BasicAer)
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import plot_histogram

Und ebenso werden einige Voreinstellungen über die Ausgabe von Grafiken und der Simulationssoftware getroffen.

In [ ]:
%matplotlib inline

In [ ]:
simulator = Aer.get_backend('qasm_simulator')

### Erinnerung
In der Vorlesung hatten wir die Operation
$$x\oplus y = (x+y)\mathrm{\, mod\,} 2$$
eingeführt, für die 
$$0\oplus 0 =0\quad\quad 0\oplus 1 = 1\quad\quad 1\oplus 0 = 1\quad\quad 1\oplus 1 = 0$$
gilt. Sie ist äquivalent zu einem ''Exklusiven Oder''.

### Nützliches
Für viele Probleme in diesem Notebook können die Wikiversity-Seiten zu [Qubits](https://de.wikiversity.org/wiki/Kurs:Quantencomputing/Qubits) und zu [Registern](https://de.wikiversity.org/wiki/Kurs:Quantencomputing/Register) sehr hilfreich sein.

## Das Problem von Deutsch
Es gibt vier Operationen, die auf ein klassisches Bit ausgeführt werden können (Dabei ist $x\in\{0, 1\}$ immer der Wert des Bits):
 * Die Identität $I$ mit $f(x)=x$
 * Der Bitflip $X$ mit $f(x) = 1\oplus x$
 * Die konstante $1$ mit $f(x)=1$ 
 * Die konstate $0$ mit $f(x)=0$

Die Operationen Identität $I$ und Bitflip $X$ sind eindeutig umkehrbar: Ist das Ergebnis $f(x)$ und die Funktion bekannt, so ist auch klar, welchen Wert $x$ das Bit vor der Operation hatte. Außerdem werden gleich viele Eingabewerte auf $0$ wie auf $1$ abgebildet. Aus diesem Grund werden diese beiden Funktionen auch als _balanciert_ bezeichnet. 

Dem gegenüber stehen die Funktionen konstant $1$ und konstant $0$, die weder eindeutig umkehrbar sind, noch gleich oft auf $0$ bzw. $1$ abbilden. Sie werden als _konstant_ bezeichnet.

Um zu unterscheiden, ob ein gegebenes, aber unbekanntes Bauteil, eine konstante oder balancierte Funktion ist, braucht ein klassischer Computer 2 Anfragen.

__Aufgabe__: Mache Dir die letzte Aussage an Hand der Funktionen von Oben klar.

__Lösung__: Bei der Identität würde bei der Eingabe $0$ die Aussage $0$ erzeugen. Diesselbe Ausgabe würde aber auch bei der Funktion konstant $0$ entstehen. Damit ist noch nicht klar, ob es sich um eine konstante oder balancierte Funktion handelt. Die gleiche Situation ergibt sich für jedes mögliche Paar von Ein- und Ausgabewerten. Es gibt immer jeweils eine konstante und eine balancierte Funktion, die in Frage kommt. Erst durch eine zweite Anfrage kann genau bestimmt werden, um welche Funktion es sich handelt.

Mit Quantencomputern ist hingegen nur __eine einzige__ Anfrage nötig. 

### Umkehrbare Operationen
Wie wir besprochen haben, müssen Operationen auf Qubits unitär und damit eindeutig umkehrbar sein. Die Operationen konstant $1$ und konstant $0$ erfüllen diese Bedingung aber __nicht__! 

Um dieses Problem zu lösen kann neben dem Eingabe-Qubit $|x\rangle$ ein Hilfs-Qubit $|h\rangle$ betrachtet werden. Wird jede Operation durch eine Funktion $f:\{0, 1\}\to\{0, 1\}, x\mapsto f(x)$ beschrieben, so lässt sich für $x, h \in \{0, 1\}$ in einem Quantenschaltkreis die Operation
$$U_f|x\rangle |h\rangle = |x\rangle |h\oplus f(x)\rangle$$
definieren.

__Fragen__:
 * Warum handelt es sich bei $U_f$ um eine umkehrbare Operation? <br>
  _Tipp_: Was passiert, wenn $U_f$ zweimal hintereinander angewandt wird?
 *  __Bonus__: Warum ist $U_f$ unitär?<br>
 * Wie viele Anfragen braucht ein klassischer Computer um mit einem Hilfsbit zwischen einer balancierten und konstanten Funktion zu unterscheiden?

__Antworten__:
 * Da $U_fU_f|x\rangle|h\rangle = U_f|x\rangle |h\oplus f(x)\rangle = |x\rangle |h\oplus f(x)\oplus f(x)\rangle$ und $f(x)\oplus f(x) = 0$ gilt, zeigt sich, dass $U_fU_f|x\rangle |h\rangle = |x\rangle |h\rangle$ ist.
 * Der Zustand $U_f|x\rangle|h\rangle=|x\rangle|h\oplus f(x)\rangle$ kommt aus der Menge $\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$ und hat damit den Betrag Eins. Eine Unitäre Matrix ist Längenerhaltend. Da die Länge der Basiselemente erhalten bleibt, bleibt auch die Länge jedes Zustandsvektors erhalten.
 *Selbst mit einem Hilfsbit braucht ein klassischer Computer noch zwei Anfragen: Bspw. kann die Kombination aus Eingabe $x = 1, h = 1$ und Ausgabe $x' = 1, h' = 0$ betrachtet werden. Diese können durch die Identität aber auch durch konstant 1 verursacht worden sein. Somit sorgt das Einführen des Hilfsbit alleine _nicht_ für eine effizientere Behandlung des Problems.

### Darstellen der Vier Operationen
Die Operation $U_f$ für die vier oben genannten Ein-Bit-Operationen lassen sich alle durch die Gatter $X$, $I$ und das CNOT-Gatter $C_X$ darstellen. (Zur Erinnerung: Das CNOT-Gatter wird durch ``cx(<qubit0>, <qubit1>)`` aufgerufen und verändert nur ``<qubit1>``.) 

__Aufgabe__: Überlege Dir, wie das gehen könnte und implimentiere sie in das Code-Gerüst.

In [ ]:
# Eine Python-Funktion, die Uf beschreibt 
# Circuit ist ein Schaltkreis mit 2 Qubits
# Dabei ist Bit 0 das Hilfsbit und Bit 1 das Bit x
def BlackBox(Circuit, functionname = "NULL"):
    Circuit.barrier()
    if functionname == "NULL": 
        # Beispielhafte Implimentierung von Uf für f(x)=0
        Circuit.id(0) 
    elif functionname == "EINS":
        # Füge hier die Operation(en) für Uf im Fall f(x) = 1 ein
        Circuit.x(0)
    elif functionname == "ID":
        # Füge hier die Operation(en) für Uf im Fall f(x) = x ein
        Circuit.cx(1, 0)
    elif functionname == "X":
        # Füge hier die Operation(en) für Uf im Fall f(x) = 1+x ein
        Circuit.cx(1, 0)
        Circuit.x(0)
    else:
        print("Da ist etwas schief gelaufen")
    Circuit.barrier()

# Hier werden die beiden Qubits als eigene Quantenregister erstellt, um sie 
# bennenen zu können
Hilfsbit = QuantumRegister(1, name = 'h')
Inputbit = QuantumRegister(1, name = 'x')

# Sie können mit QuantumCircuit zu einem Schaltkreis zusammengefasst werden
circuit = QuantumCircuit(Hilfsbit, Inputbit)

# Nun werden die Schaltkreise für jede mögliche Funktion f(x) angezeigt

# f(x) = 0
BlackBox(circuit, "NULL")

# f(x) = 1
BlackBox(circuit, "EINS")

# f(x) = x
BlackBox(circuit, "ID")

# f(x) = 1+x
BlackBox(circuit, "X")

# Schlussendlich wird der Schaltkreis gezeichnet
circuit.draw(output = 'mpl')    

Um die Umsetzung der Funktion $U_f$ zu überprüfen, kann folgender Code verwendet werden.
 * Zuerst wird dier Funktion ``SETUP`` definiert, die einen Schaltkreis und einen Anfangszustand zweier Qubits entgegennimmt. Sie versetzt die Qubits 0 und 1 des übergebenen Quantenschaltkreises in den übergebenenn Anfangszustand 
 * Die zweite Zelle erstellt einen Quantenschaltkreis mit zwei Qubits. Durch das variieren des Arguments von ``SETUP`` können verschiedenen Anfangszustände durchprobiert werden. Ebenso kann die obige Funktion ``BlackBox`` mit den verschiedenen Spezifikatoren für $f(x)$ aufgerufen werden. Durch die anschließende Simulation lässt sich prüfen, ob sich die richtigen Zustände ergeben

In [ ]:
def SETUP(Circuit, inputstate = "00"):
    Circuit.reset(range(2))  # Resete die beiden Bits
    if inputstate == "00":  # Die Bits sind schon resetet
        return
    elif inputstate == "01": # Bit 0 muss umgedreht werden
        Circuit.x(0)
    elif inputstate == "10": # Bit 1 muss umgedreht werden
        Circuit.x(1)
    elif inputstate == "11": # Beide Bits müssen umgedreht werden
        Circuit.x(range(2))
    else:
        print("Da ist wohl etwas in SETUP schief gelaufen")

In [ ]:
# Quantenschaltkreis mit zwei Qubits und zwei klassischen Bits
qc = QuantumCircuit(2, 2)

# Einstellen des Anfangszustands
SETUP(qc, "00")

# Anwenden der oben definierten Funktionen
BlackBox(qc, "NULL")

# Messen der Qubits und schreiben in die klassischen Bits
qc.measure(range(2), range(2))

# Simuulieren des Schaltkreises
job = execute(qc, simulator, shots = 1000)
results = job.result()
plot_histogram(results.get_counts())

### Der Algorithmus von Deutsch
Betrachte den Quantenschaltkreis, der durch den folgenden Code erzeugt erzeugt wird.

In [ ]:
# Hier werden die beiden Qubits als eigene Quantenregister erstellt, um sie 
# bennenen zu können
Hilfsbit = QuantumRegister(1, name = 'h')
Inputbit = QuantumRegister(1, name = 'x')

# Ebenso kännen klassische Bits erstellt werden
KlassischeBits = ClassicalRegister(1, name = 'c')

# Alle Register werden zu eimem Quantenschaltkreis zusammengefasst
circuit = QuantumCircuit(Hilfsbit, Inputbit, KlassischeBits)

# Alle Qubits werden auf Null initialisiert
circuit.reset(range(2))

# Das Hilfsbit wird in den Zustand |1> versetzt
circuit.x(0)

# Die Barriere sorgt für Übersichtlichkeit
circuit.barrier()

# Auf beide Qubits wird die Hadamard-Transformation angewandt
circuit.h(range(2))

# Der Schaltkreis wird gezeichnet
circuit.draw(output = 'mpl', plot_barriers = False)

__Aufgabe__: Berechne in welchem Zustand sich das Quantenregister $|R_2\rangle = |x\rangle|h\rangle$ nach der Ausführung dieses Schaltkreieses befindet.

__Lösung__: Zu Anfang befindet sich das Register in dem Zustand $|R_2\rangle = |0\rangle|0\rangle$. Danach wird das Hilfsbit umgedreht, so dass der neue Zustand $|R_2'\rangle=|0\rangle|1\rangle$ ist. Anschließden wird auf beide Zustände die Hadamard-Matrix angewandt. Die Hadamard-Matrix hat auf die Zustände die Wirkung
$$H|0\rangle=|+\rangle=\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$$
und 
$$H|1\rangle=|-\rangle=\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$$
Damit befindet sich das Register am Ende im Zustand
$$|R_2''\rangle=|+\rangle|-\rangle=\frac{1}{2}(|00\rangle-|01\rangle+|10\rangle-|11\rangle)\\
=\frac{1}{2}\bigg(|0\rangle(|0\rangle-|1\rangle)+|1\rangle(|0\rangle-|1\rangle)\bigg)$$

__Aufgabe__: In welchem Zustand befindet sich das Register $|x\rangle|h\rangle$ wenn nun zusätzlich $U_f$ angewandt wird?<br>
_Hinweis_: Warum ist 
$$|0\oplus f(x)\rangle-|1\oplus f(x)\rangle=(-1)^{f(x)}(|0\rangle-|1\rangle)$$
für ein beliebiges $x\in\{0, 1\}$ gültig?

__Lösung__: Durch das Anwenden von $U_f$ entsteht der Zustand
$$|R_2'''\rangle= \frac{1}{2}\bigg(|0\rangle(|0\oplus f(0)\rangle-|1\oplus f(0)\rangle)+|1\rangle(|0+\oplus f(1)\rangle-|1\oplus f(1)\rangle)\bigg)$$
Für die Kombination 
$$|0\oplus f(x)\rangle-|1\oplus f(x)\rangle$$
kann aber der Ausdruck
$$(-1)^{f(x)}(|0\rangle-|1\rangle)$$
gefunden werden. Denn wenn $f(x)=0$ ist, so ist der Ausdruck gerade $|0\rangle-|1\rangle$. Ist hingegen $f(x)=1$, so ist der Ausdruck $|1\rangle-|0\rangle=(-1)\cdot(|0\rangle-|1\rangle)$. Da $(-1)^0=1$ ist, kann in beiden Fällen der besagte Ausdruck geschrieben werden.
Dmit kann der Zustand auf 
$$|R_2'''\rangle = \frac{1}{2}\bigg(|0\rangle(-1)^{f(0)}(|0\rangle-|1\rangle)+(-1)^{f(1)}(|0\rangle-|1\rangle)\bigg)\\
=\frac{1}{\sqrt{2}}\bigg((-1)^{f(0)}|0\rangle+(-1)^{f(1)}|1\rangle\bigg)|-\rangle$$
umgeformt werden. Da $(-1)^{f(x)+f(x)}=(-1)^{f(x)-f(x)}=(-1)^{f(x)\oplus f(x)}=1$ gilt und globale Phasenfaktoren den gleichen Zustand beschreiben, kann dieser Zustand auch als
$$|R_2'''\rangle=\frac{(-1)^{f(0)}}{\sqrt{2}}\bigg(|0\rangle+(-1)^{f(0)\oplus f(1)}|1\rangle\bigg)|-\rangle$$
geschrieben werden. Damit wurde jetzt die Auswirkung der Funktion $f(x)$ in ein relatives Vorzeichen verlagert. Relative Vorzeichen (bzw. Phasenunterschiede) in Quantenzuständen sind immer für Interferrenzeffekte verantwortlich. Und solche Interferrenzeffekte sind messbar.

__Aufgabe__: Was für eine Operation kann nun auf das Bit $|x\rangle$ angewendet werden, um bei einer anschließenden Messung dieses Bits zu unterscheiden, ob es sich um eine konstante oder balancierte Funktion handelt?

__Lösung__: Wird auf das Bit $|x\rangle$ im obigen Zustand
$$|\psi\rangle=\frac{(-1)^{f(0)}}{\sqrt{2}}(|0\rangle+(-1)^{f(0)\oplus f(1)}|1\rangle)$$
die Hadamard-Transformation $H$ angewendet, so wird sich der Zustand
$$H|\psi\rangle=\frac{(-1)^{f(0)}}{\sqrt{2}}\bigg(|+\rangle+(-1)^{f(0)\oplus f(1)}|-\rangle\bigg)\\
=\frac{(-1)^{f(0)}}{2}\bigg(|0\rangle+|1\rangle+(-1)^{f(0)\oplus f(1)}(|0\rangle-|1\rangle)\bigg)\\
=\frac{(-1)^{f(0)}}{2}\bigg(|0\rangle\left(1+(-1)^{f(0)\oplus f(1)}\right)+|1\rangle\left(1-(-1)^{f(0)\oplus f(1)}\right)\bigg)$$
ergeben.
* Ist $f(x)$ eine konstante Funktion $f(0)=f(1)$ bzw. $f(0)\oplus f(1)=0$, so befindet sich das Bit $|x\rangle$ am Ende der Berechnung im Zustand $|0\rangle$. 
* Ist $f(x)$ hingegen balanciert, so dass $f(0)=1\oplus f(1)$ bzw. $f(0)\oplus f(1)=1$ gilt, so befindet sich das Bit $|x\rangle$ am Ende der Berechnung im Zustand $|1\rangle$.

### Implementierung des Algorithmus

__Aufgabe__: Implementiere den Algorithmus in dem folgenden Code-Gerüst.

In [ ]:
# Hier werden die beiden Qubits als eigene Quantenregister erstellt, um sie 
# bennenen zu können
Hilfsbit = QuantumRegister(1, name = 'h')
Inputbit = QuantumRegister(1, name = 'x')

# Ebenso kännen klassische Bits erstellt werden
KlassischeBits = ClassicalRegister(1, name = 'c')

# Alle Register werden zu eimem Quantenschaltkreis zusammengefasst
circuit = QuantumCircuit(Hilfsbit, Inputbit, KlassischeBits)

# Alle Qubits werden auf Null initialisiert
circuit.reset(range(2))

# Das Hilfsbit wird in den Zustand |1> versetzt
circuit.x(0)

# Die Barriere sorgt für Übersichtlichkeit
circuit.barrier()

# Auf beide Qubits wird die Hadamard-Transformation angewandt
circuit.h(range(2))

# Füge hier die fehlenden Operationen ein, um zwischen balancierten 
# und konstanten Funktionen zu unterscheiden

# Zunächst muss die Funktion angewandt werden
BlackBox(circuit, "X")

# Es muss die Hadamard-Transformation auf das Inputbit (1) angewandt werden
circuit.h(1)

# Es muss das Inputbit gemessen werden
circuit.measure(1, 0)

# Der Schaltkreis wird gezeichnet
circuit.draw(output = 'mpl')

Mit der folgenden Code-Zelle, kannst Du ausprobieren, ob der Schaltkreis seine Aufgabe auch wirklich erfüllt.

In [ ]:
job = execute(circuit, simulator, shots = 1000)
results = job.result()
plot_histogram(results.get_counts())

## Das Problem von Deutsch-Josza
Nun kann das Problem von Deutsch verallgemeinert werden. Die Idee ist, dass eine Funktion $f$ betrachtet wird, die $n$ Bits entgegen nimmt und diese entweder auf $0$ oder $1$ abbildet. Sie kann also als $f:\{0, 1\}^n\to\{0, 1\}, x\mapsto f(x)$ geschrieben werden. Auch hier geht es wieder um die Frage, wie sich erkennen lässt, ob die Funktion nur konstante Funktionswerte annimmt oder ob sich gleich oft das Ergebnis $0$ sowie $1$ ergibt. 

__Aufgabe__: Wie viele Zugriffe braucht ein klassischer Computer im schlechteseten Fall, um zwischen einer konstanten und einer balancierten Funktion unterscheiden zu können?

__Lösung__: Es gibt $2^n$ mögliche Eingaben. Das bedeutet, bei einer balancierten Funktion werden je $2^{n-1}$ Eingaben auf $0$ bzw. auf $1$ abgebildet. Im schlechtesten Fall werden gerade die Eingaben ausprobiert, die alle den selben Wert ergeben. Daher müssen im schlechtesten Fall $2^{n-1}+1$ Eingaben geprüft werden. Ergibt sich bei der letzten Eingabe der gleiche Wert, wie zuvor, so ist die Funktion konstant. Ergibt sich ein anderer Wert, so ist die Funktion balanciert.

### Umkehrbare Operationen und Darstellungen dieser

Genau wie bei dem Problem von Deutsch, muss auch hier eine umkehrbare Variante der Funktion verwendet werden, die durch die Transformation 
$$U_f|X\rangle|h\rangle=|X\rangle|h\oplus f(X)\rangle$$
eingeführt werden, wobei $X\in\{0, 1\}^n$.

Wird das Register als $|X\rangle|h\rangle$ geschrieben, so ist der Vektor des $\mathbb{C}^{2^{n+1}}$ und durch das Tensorprodukt $|X\rangle\otimes|h\rangle$ dargestelllt, bei dem $|X\rangle$ aus dem $\mathbb{C}^{2^n}$ kommt. Das bedeutet, dass der Vektor die Form
$$|X\rangle\otimes|h\rangle=\begin{pmatrix}
C_{0\dots 00}\\
C_{0\dots 01}\\
\vdots\\
C_{1\dots 1}
\end{pmatrix}\otimes\begin{pmatrix}
\alpha_h\\
\beta_h
\end{pmatrix}=\begin{pmatrix}
C_{0\dots 00}\alpha_h\\
C_{0\dots 00}\beta_h\\
C_{0\dots 01}\alpha_h\\
C_{0\dots 01}\beta_h\\
\vdots\\
C_{1\dots 11}\alpha_h\\
C_{1\dots 11}\beta_h
\end{pmatrix}
$$
annimmt. Da bisher klassische Bits angenommen werden ist $C_X, \alpha_h\beta_h\in\{0, 1\}$. 

Dieser Vektor lässt sich in Zweierblöcke aufteilen. Der erste Teil $C_{X}$ beschreibt immer den Wert des Registers $X$ während die letzte Stelle jeweils den Wert von $h$ beschreibt. Das hat den Vorteil, dass sich $U_f$ als eine Matrix bestehend aus Blöcken von $2\times 2$ Matrizen auf der Diagonale beschreiben lässt, von denen jede $2\times 2$ Matrix beschreibt, wie sich beim Anwenden der Opartion $U_f$ der Vektorblock 
$$
\begin{pmatrix}
C_X\alpha_h\\
C_X\beta_h
\end{pmatrix}
$$
ändert.

__Aufgabe__: Wie sehen die $2\times 2$ Matrizen auf der Diagonale aus, wenn $f(X)=0$ bzw. $f(X)=1$ gilt?

__Lösung__: Ist $f(X)=0$, so wird $|h\rangle$ auf $|h\oplus f(x)\rangle=|h\oplus 0\rangle=|h\rangle=I|h\rangle$ gesetzt. Das bedeutet der Vektor ändert sich nicht und der Matrixblock wird durch $I$ ausgedrückt.<br>
Ist nun hingen $f(X)=1$, so wird $|h\rangle$ auf $|h\oplus f(x)\rangle=|h\oplus 1\rangle=X|h\rangle$ gesetzt. Damit entspricht der Zweierblock gerade dem Bitflip $X$.

Im Folgenden werden zwei Beispielfunktionen als balanciert und konstant für den späteren Gebrauch implementiert. Es wird dann die Matrix $U_f$ bestimmt und in ein Bauteil für einen Quantenschaltkreis umgewandelt.

In [ ]:
# n ist die Anzahl der Qubits im Register X

def func_const(x, n):    # Beispiel einer konstanten Funktion
    return 1


def func_balanced(x, n):    # Beispiel einer balancierten Funktion
    if x < 2**(n-1):
        return 0    # Die erste Hälfte der Eingabe wird auf Null abgebildet
    else:
        return 1    # Die zweite Hälfte der Eingabe wird auf Eins abgebildet
    

def Matrix_U(func, n):    # Implementierung der Matrix Uf
    # Erzeuge eine Matrix der richtigen Größe gefüllt mit Nullen
    U = [[0 for i in range(2**(n+1))] for k in range(2**(n+1))] 
    for x in range(2**n):
        if func(x, n)==0:
            # f(x)=1 also muss auf der Diagonale die Einheitsmatrix liegen
            U[2*x][2*x] = 1
            U[2*x+1][2*x+1] =1
        else:
            # f(x)=1 also muss ein Bit-Flip auf der Diagonale liegen
            U[2*x][2*x+1] = 1
            U[2*x+1][2*x] = 1
    # Der Ausgabewert der Funktion ist die Matrix U
    return U


def Uf_on_QuantumCircuit(Circuit, n, typ = 'balanced'):
    if typ == 'balanced':    # Eine balancierte Funktion soll verwendet werden
        U = Matrix_U(func_balanced, n)    # Bestimmt die Matrix U der Funktion
        Oracle = Operator(U)    # Operator kann Matrizen in Bauteile umwandeln
        Circuit.append(Oracle, range(n+1))    # Fügt das Bauteil in den Schaltkreis 
    elif typ == 'const':    # Eine konstante Funktion soll verwendet werden
        U = Matrix_U(func_const, n)    # Bestimmt die Matrix U der Funktion
        Oracle = Operator(U)    # Operator kann Matrizen in Bauteile umwandeln
        Circuit.append(Oracle, range(n+1))    # Fügt das Bauteil in den Schaltkreis 
    else:
        print("Da ist etwas schief gelaufen in Uf_on_QuantumCircuit!")

### Implementierung des Algorithmus
__Aufgabe__: Überlege Dir, wie der Algorithmus von Deutsch verallgemeinert werden kann, um das Problem von Deutsch-Josza zu lösen. Implementiere den Algorithmus in das folgende Code-Gerüst

In [ ]:
# Anzahl der Qubits im Register X
n = 2

# Erstellen der einzelnen Register
X = QuantumRegister(n, name = 'x')
H = QuantumRegister(1, name = 'h')
# Wie viele klassische Bist werden benötigt
n_klassisch = n # 
C = ClassicalRegister(n_klassisch, name = 'c')

# Ertsllen des Quantenschaltkreises
# Das Hilfsbit ist hierbei Bit 0
circuit = QuantumCircuit(H, X, C)

# Verallgemeinere nun den Algorithmus von Deutsch
# Initialisiere alle Zustände, das Registe X auf |0...0>, das Hilfsbit auf |1>
circuit.reset(range(n+1))
circuit.x(0)

circuit.barrier()

# Die Hadamard-Transformation wird auf alle Qubits angewendet
circuit.h(range(n+1))

circuit.barrier()

# Die Funktion Uf wird angewendet
Uf_on_QuantumCircuit(circuit, n, typ = 'const')

# Es wird eine Hadamard-Transformation auf das Register X ausgeführt
circuit.h(range(1, n+1))

circuit.barrier()

# Es wird eine Messung des Registers X durchgeführt und 
# in die klassischen Bits geschrieben
circuit.measure(range(1, n+1), range(n))


# Zeichne den Quantenschaltkreis
circuit.draw(output = 'mpl', plot_barriers = False)

Mit dem folgenden Code wird der Schaltkreis ausgeführt und es kann geprüft werden, ob er die richtigen Ergebnisse liefert.

In [ ]:
job = execute(circuit, simulator, shots = 1000)
results = job.result()
plot_histogram(results.get_counts())

## Erklärung des Algorithmus
__Extra-Aufgabe__: Zeige, dass sich bei der Messung der Zustand $|0\dots 0\rangle$ nur dann einstellen kann, wenn es sich um eine konstante Funktion handelt.<br>
_Tipp_: Hier ist die Formel zur Anwendung der Hadamard-Transformation $H_n$ auf einen beliebigen Zustand aus unserem [Wikiversity-Artikel zu Registern](https://de.wikiversity.org/wiki/Kurs:Quantencomputing/Register) hilfreich.

__Lösung__:
Nach dem Anwenden der Hadamard-Transformationen befindet sich das System im Zustand
$$|X\rangle|h\rangle = \left(\frac{1}{\sqrt{2^n}}\sum_{Z\in\{0, 1\}^n}|Z\rangle\right)\otimes |-\rangle$$
Wie auch im Algorithmus von Deutsch ergibt sich nach dem Anwenden von $U_f$ für jedes $Z$ der Ausdruck
$$\frac{1}{\sqrt{2}}\left(|Z\rangle|0\oplus f(Z)\rangle-|Z\rangle|1\oplus f(Z)\rangle\right)=\frac{(-1)^{f(z)}}{\sqrt{2}}|Z\rangle(|0\rangle-|1\rangle)\\
=(-1)^{f(Z)}|Z\rangle|-\rangle$$
und somit ergibt sich nach der Anwendung von $U_f$ der Zustand
$$|X\rangle|h\rangle = \left(\frac{1}{\sqrt{2^n}}\sum_{Z\in\{0, 1\}^n}(-1)^{f(Z)}|Z\rangle\right)\otimes |-\rangle$$
Am Zustand $|h\rangle$ wird nichts mehr verändert, weshalb er nicht mehr betrachtet werden muss. Nach der Anwendung der Hadamard-Matrizen auf $|X\rangle$ kann der Zustand
$$H_n|X\rangle = H_n\frac{1}{\sqrt{2^n}}\sum_{Z\in\{0, 1\}^n}(-1)^{f(Z)}|Z\rangle\\
=\frac{1}{2^n}\sum_{W\in\{0, 1\}^n}\sum_{Z\in\{0, 1\}^n}(-1)^{f(Z)}(-1)^{W\cdot Z}|W\rangle$$
gefunden werden.
Soll der Zustand $|0\dots 0\rangle$ gemessen werden, muss die Projektion auf diesen betrachtet werden. Damit kann
$$\langle 0\dots 0|X\rangle = \frac{1}{2^n}\sum_{Z\in\{0, 1\}^n}(-1)^{f(Z)}$$
gefunden werden, das $W$ auf $0\dots 0$ gesetzt werden muss. Ist $f(Z)$ konstant, so wird $2^n$ mal der gleiche Term aus $\{-1, 1\}$ aufaddiert. Das bedeutet die Projektion ergibt $\pm 1$ und das Betragsquadrat davon ist $1$. Also wird bei einer konstanten Funktion mit $100\%$-ger Wahrscheinlichkeit der Zustand $|0\dots 0\rangle$ gemssen. Ist die Funktion hingegen balanciert, so wird $2^{n-1}$ mal der Wert $+1$ und $2^{n-1}$ mal der Wert $-1$ addiert. Insgesamt wird die Projektion also Null und die Wahrscheinlichkeit den Zustand $|0\dots 0\rangle$ zu messen beträgt Null. Wichtig ist hierbei, dass der Zustand der dann gemessen wird, von der genauen Form von $f$ abhängig ist.

## Der Algorithmus von Bernstein-Vazirani
Eine weitere Anwendung für den Schaltkreis, der das Problem von Deutsch-Josza löst, besteht im Problem von Bernstein-Vazirani. Dabei wird eine lineare Funktion
$$f: \{0, 1\}^n\to \{0, 1\}, x\mapsto f(x)=a\cdot x= \left(\sum_{k=0}^{n-1}a_k\cdot x_k\right)\mathrm{\, mod\, }2$$
betrachtet. Darin ist $a\in\{0, 1\}^n$. Im Folgenden soll untersucht werden, was der Algorithmus von Deutsch-Josza bei solch einer Funktion für ein Ergebnis liefert.

### Bestimmen der Matrix $U_f$
Um den Algorithmus von Deutsch-Josza anwenden zu können, muss erst $U_f$ für die obige Funktion implementiert werden. 

__Aufgabe__: Vervollständige das untenstehende Code-Gerüst, das $U_f$ implementiert.

In [ ]:
# Da x als Dezimalzahl vorliegt, muss es erstmal in eine Binärzahl umgewandelt werden
def dec_to_bin(x, n):
    x=int(x)
    X = np.zeros(n, dtype = int)    # Eine Liste (numpy-Array) mit n Nullen. Sie stellt die Binärdarstellung dar, dtype = int sorgt dafür, dass nur ganze Zahlen (0, 1) in der Liste stehen
    for k in range(n):
        X[n-k-1] = x%2   # % ist in Python Modulo, also x mod 2 wird an die aktuelle Stelle geschrieben
        x = x/2    # / ist in Python für Integer Division ohne Rest, das heißt, das "Komma" wird ein Stelle weiter geschoben
    return X    # Die Liste mit der Binärdarstellung von X wird zurückgegeben
     
    
def func_linear(x, n, a):
    # x ist eine Zahl zwischen 0 und 2^n-1
    # n ist die Anzahl der Qubits im Register X
    # a ist der Vektor a, der als Numpy Array der Länge n vorliegt (Binärdarstellung)
    
    # Welche Operationen müssen ausgeführt werden, damit das 
    # Ergebnis f(x)=ax von oben ausgegeben wird?
    # Tipp: Werden zwei Numpy Arrays multipliziert passiert 
    #       das für jeden Eintrag einzeln
    #       Um alle Einträge eines Numpy Arrays aufzusummieren 
    #       gibt es den Befehl np.sum(<array>)
    
    X = dec_to_bin(x, n)
    return np.sum(X*a)%2


def Uf_linear(Circuit, n, a):
    # Erstelle eine Matrix der richtigen Größe voller Nullen
    U = [[0 for i in range(2**(n+1))] for k in range(2**(n+1))] 
    for x in range(2**n):
        if func_linear(x, n, a)==0:
            # f(x)=1 also muss auf der Diagonale die Einheitsmatrix liegen
            U[2*x][2*x] = 1
            U[2*x+1][2*x+1] =1
        else:
            # f(x)=1 also muss ein Bit-Flip auf der Diagonale liegen
            U[2*x][2*x+1] = 1
            U[2*x+1][2*x] = 1
    Oracle = Operator(U)    # Operator kann Matrizen in Bauteile umwandeln
    Circuit.append(Oracle, range(n+1)) # Fügt das Bauteil in den Schaltkreis ein
    

# Teste die Funktion:
# Mit den Argumenten 4, 3, np.array([1, 1, 1]) sollte sich 1 ergeben
# Mit den Argumenten 3, 3, np.array([1, 1, 1]) sollte sich 0 ergeben
func_linear(3 , 3, np.array([1, 1, 1]))

### Implementieren des Schaltkreises
__Aufgabe__: Implementiere erneut den Schaltkreis vom Problem von Deutsch-Josza, diesmal mit dem Bauteil für die lineare Funktion in dem nachfolgenden Code-Gerüst.

In [ ]:
# Anzahl der Qubits im Register X
n = 5

# Ergänze das Code-Gerüst, um das Problem von Bernstein-Vazirani zu lösen
# Gib den Vektor a dabei durch np.array([a(n-1) , ..., a3, a2, a1, a0]) an.

# Erstellen der einzelnen Register
X = QuantumRegister(n, name = 'x')
H = QuantumRegister(1, name = 'h')
C = ClassicalRegister(n, name = 'c')

# Ertsllen des Quantenschaltkreises
# Das Hilfsbit ist hierbei Bit 0
circuit = QuantumCircuit(H, X, C)

# Initialisiere alle Zustände, das Registe X auf |0...0>, das Hilfsbit auf |1>
circuit.reset(range(n+1))
circuit.x(0)

circuit.barrier()

# Die Hadamard-Transformation wird auf alle Qubits angewendet
circuit.h(range(n+1))

circuit.barrier()

# Die Funktion Uf linear wird angewendet
Uf_linear(circuit, n, np.array([1, 0, 1, 0, 1]))

# Es wird eine Hadamard-Transformation auf das Register X ausgeführt
circuit.h(range(1, n+1))

circuit.barrier()

# Es wird eine Messung des Registers X durchgeführt und 
# in die klassischen Bits geschrieben
circuit.measure(range(1, n+1), range(n))


# Zeichne den Quantenschaltkreis
circuit.draw(output = 'mpl', plot_barriers = False)

### Erklärung des Algorithmus

__Aufgabe__: Wie wahrscheinlich ist es am Ende den Zustand $|X\rangle = |a\rangle$ zu messen?

__Lösung__: Bereits bei der Erklärung zum Algorithmus von Deutsch-Josza hat sich gezeigt, dass sich das Quantenregister $|X\rangle$ vor der Messung im Zustand
$$|X\rangle = \frac{1}{2^n}\sum_{W\in\{0, 1\}^n}\sum_{Z\in\{0, 1\}^n}(-1)^{f(Z)}(-1)^{W\cdot Z}|W\rangle$$
befindet. Soll nun der Zustand $|a\rangle$ gemessen werden, so bleibt in der Summe nur der Beitrag mit $W=a$ übrig, so dass sich
$$\langle a|X\rangle=\frac{1}{2^n}\sum_{Z\in\{0, 1\}^n}(-1)^{f(Z)}(-1)^{a\cdot Z}$$
ergibt. Da nun aber $f(Z)=a\cdot Z$ ist, taucht in der Summe insgesamt der Faktor 
$$(-1)^{f(Z)}(-1)^{f(Z)}=(-1)^{f(Z)\oplus f(Z)}=(-1)^0 = 1$$
auf. Es wird also $2^n$ mal über 1 summiert und somit ist die Projektion, wie auch die Wahrscheinlichkeit gerade 1. Mit anderen Worten: Die Messung wird als Ergebnis immer den Vektor $a$ ergeben und das mit nur __einem einzigen__ Aufruf des Bauteils für die Funktion $f(x)$.

### Testen des Algorithmus

Mit dem folgenden Code kannst Du ausprobieren, was sich bei dem Ausführen des Algorithmus ergibt.

In [ ]:
job = execute(circuit, simulator, shots = 1000)
results = job.result()
plot_histogram(results.get_counts())